<a href="https://colab.research.google.com/github/sahithikodali1/Summarization-of-Biomedical-evidence/blob/Data_processing/BIOASQ_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os

print(os.listdir("/content/drive/MyDrive/Thesis_B"))

['rouge_8b_mod.csv', '.ipynb_checkpoints', 'test_data.csv', 'train_data.csv']


In [2]:
DATA_DIR = "/content/drive/MyDrive/Thesis_B"

file = pd.read_csv(os.path.join(DATA_DIR,'rouge_8b_mod.csv'))
f_values = file.values
# print(f_values[:10])
# print(col_names[:10])
q_id_split = np.split(f_values, np.where(np.diff(f_values[:,0]))[0]+1)
# print(q_id_split[:2])

In [3]:
train_data_size = round(len(q_id_split)*0.8)
test_data_size = round(len(q_id_split) - train_data_size)
print(train_data_size)
print(test_data_size)

2594
648


In [4]:
import random
random.seed(3007)
random.shuffle(q_id_split)
print(type(q_id_split[:1]))

<class 'list'>


In [5]:
train_data_beforeval = q_id_split[:2594]
f_values_train_beforeval = np.concatenate(train_data_beforeval, axis=0)
# print((f_values_train_beforeval))

q_id_split_train = np.split(f_values_train_beforeval, np.where(np.diff(f_values_train_beforeval[:,0]))[0]+1)
print(len(q_id_split_train))

2594


In [6]:
val_data_size = round(len(q_id_split_train)*0.2)
train_data_size = round(len(q_id_split_train) - val_data_size)
print(train_data_size)
print(val_data_size)

2075
519


In [7]:
train_data = q_id_split_train[:2075]
val_data = q_id_split_train[2075:]
test_data = q_id_split[2594:]
f_values_train = np.concatenate(train_data, axis=0)
f_values_val = np.concatenate(val_data, axis=0)
f_values_test = np.concatenate(test_data, axis=0)
col_names = file.columns
print(col_names)

train_df = pd.DataFrame(f_values_train, columns = col_names)
val_df = pd.DataFrame(f_values_val, columns = col_names)
test_df = pd.DataFrame(f_values_test, columns = col_names)

Index(['qid', 'pubmedid', 'sentid', 'N1', 'N2', 'L', 'S4', 'SU4',
       'sentence text', 'SU4_labels'],
      dtype='object')


In [8]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

print(len(train_df))
print(len(val_df))
print(len(test_df))
print(train_df.columns.values)

2075
519
648
33435
8148
10161
['qid' 'pubmedid' 'sentid' 'N1' 'N2' 'L' 'S4' 'SU4' 'sentence text'
 'SU4_labels']


In [9]:
# install
!pip install pytorch-pretrained-bert pytorch-nlp

# BERT imports
import torch

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification, BertForMaskedLM
from tqdm import tqdm, trange

import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [11]:
def data_grouplists(data):
  data_list = data.groupby('qid', as_index=False)['SU4','SU4_labels','sentence text'].agg(list)
  return data_list

In [12]:
train_list = data_grouplists(train_df)
val_list = data_grouplists(val_df)
test_list = data_grouplists(test_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [13]:
def add_specialtokens(data_list):
  specialtok_sentences = []

  for sentence in data_list['sentence text']:
    each_specialtoken = []
    for each in sentence:
      each_specialtoken.append(["[CLS] " + each + " [SEP]" ])
    specialtok_sentences.append(each_specialtoken)
  return specialtok_sentences

In [14]:
train_specialtok = add_specialtokens(train_list)
val_specialtok = add_specialtokens(val_list)
test_specialtok = add_specialtokens(test_list)

In [15]:
print(train_specialtok[0])
print(train_specialtok[1])

[['[CLS] Papilin is an extracellular matrix glycoprotein [SEP]'], ['[CLS] apilins are homologous, secreted extracellular matrix proteins which share a common order of protein domains. [SEP]'], ['[CLS] Both MIG-6 isoforms have a predicted N-terminal papilin cassette [SEP]'], ['[CLS] The TSR superfamily is a diverse family of extracellular matrix and transmembrane proteins, many of which have functions related to regulating matrix organization, cell-cell interactions and cell guidance. [SEP]'], ['[CLS]  Collagen IV, laminin, glutactin, papilin, and other extracellular matrix proteins were made primarily by hemocytes and were secreted into the medium. [SEP]'], ['[CLS] A sulfated glycoprotein was isolated from the culture media of Drosophila Kc cells and named papilin. [SEP]'], ['[CLS] Using expression analysis, we identify three genes that are transcriptionally regulated by HLH-2: the protocadherin cdh-3, and two genes encoding secreted extracellular matrix proteins, mig-6/papilin and him

In [16]:
def tokenize_sentences(specialtok_sentences):
  tokenized_sentences = []

  for sentence in specialtok_sentences:
    eachsent_tokenized = []
    for each in sentence:
      eachsent_tokenized.append(tokenizer.tokenize(each[0]))
    tokenized_sentences.append(eachsent_tokenized)
  return tokenized_sentences

In [17]:
train_tokenized = tokenize_sentences(train_specialtok)
val_tokenized = tokenize_sentences(val_specialtok)
test_tokenized = tokenize_sentences(test_specialtok)


In [18]:
def data_tokenizedsent(tokenized_sentences, data_list):
  data_list['tokenized_sentences'] = tokenized_sentences
  return data_list

In [19]:
print(train_tokenized[0])

[['[CLS]', 'pa', '##pi', '##lin', 'is', 'an', 'extra', '##cellular', 'matrix', 'g', '##ly', '##co', '##pro', '##tein', '[SEP]'], ['[CLS]', 'api', '##lins', 'are', 'homo', '##log', '##ous', ',', 'secret', '##ed', 'extra', '##cellular', 'matrix', 'proteins', 'which', 'share', 'a', 'common', 'order', 'of', 'protein', 'domains', '.', '[SEP]'], ['[CLS]', 'both', 'mig', '-', '6', 'iso', '##forms', 'have', 'a', 'predicted', 'n', '-', 'terminal', 'pa', '##pi', '##lin', 'cassette', '[SEP]'], ['[CLS]', 'the', 'ts', '##r', 'superfamily', 'is', 'a', 'diverse', 'family', 'of', 'extra', '##cellular', 'matrix', 'and', 'trans', '##me', '##mb', '##rane', 'proteins', ',', 'many', 'of', 'which', 'have', 'functions', 'related', 'to', 'regulating', 'matrix', 'organization', ',', 'cell', '-', 'cell', 'interactions', 'and', 'cell', 'guidance', '.', '[SEP]'], ['[CLS]', 'col', '##lage', '##n', 'iv', ',', 'lam', '##ini', '##n', ',', 'g', '##lu', '##ta', '##ct', '##in', ',', 'pa', '##pi', '##lin', ',', 'and', 'o

In [20]:
data_train_tokenized = data_tokenizedsent(train_tokenized, train_list)
data_val_tokenized = data_tokenizedsent(val_tokenized, val_list)
data_test_tokenized = data_tokenizedsent(test_tokenized, test_list)

In [21]:
print(data_train_tokenized.columns.values)
print(data_train_tokenized)

['qid' 'SU4' 'SU4_labels' 'sentence text' 'tokenized_sentences']
       qid  ...                                tokenized_sentences
0        2  ...  [[[CLS], pa, ##pi, ##lin, is, an, extra, ##cel...
1        4  ...  [[[CLS], os, ##te, ##op, ##rot, ##eger, ##in, ...
2        6  ...  [[[CLS], serum, mir, -, 132, ,, mir, -, 26, ##...
3        7  ...  [[[CLS], p, ##yr, ##ido, ##sti, ##gm, ##ine, i...
4        9  ...  [[[CLS], w, ##nt, signaling, activate, ##s, th...
...    ...  ...                                                ...
2070  3235  ...  [[[CLS], the, genus, ana, ##pl, ##as, ##ma, be...
2071  3236  ...  [[[CLS], a, number, of, bacteria, use, a, clas...
2072  3237  ...  [[[CLS], ri, ##tu, ##xi, ##ma, ##b, in, childr...
2073  3238  ...  [[[CLS], the, crystal, structures, of, pi, ##m...
2074  3239  ...  [[[CLS], our, results, suggest, that, m2, -, l...

[2075 rows x 5 columns]


In [22]:
print(len(train_tokenized[0]))
print(len(train_tokenized[1]))
print(len(train_tokenized[2]))

11
13
32


In [23]:
def token2ids(tokenized_sentences):
  # Set the maximum sequence length. 
  MAX_LEN = 512
  token_ids = []

  for tokenized_sentence in tokenized_sentences:
    tokens_to_ids = [tokenizer.convert_tokens_to_ids(each) for each in tokenized_sentence]
    tokens_to_ids = pad_sequences(tokens_to_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    token_ids.append(tokens_to_ids)
  return token_ids

In [24]:
train_tokens2ids = token2ids(train_tokenized)
val_tokens2ids = token2ids(val_tokenized)
test_tokens2ids = token2ids(test_tokenized)

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (529 > 512). Running this sequence through BERT will result in indexing errors


In [25]:
print(len(train_tokens2ids))
print(len(train_tokens2ids[0]))
print(len(train_tokens2ids[1]))

2075
11
13


In [72]:
def create_masks(token_ids):
  attention_masks = []

  for tid in token_ids:
    eachtid_mask = []
    for each in tid:
      mask = [float(i>0) for i in each] #for each token 
      eachtid_mask.append(mask)#for each sentence
    attention_masks.append(np.array(eachtid_mask))
  return attention_masks

In [30]:
#define the model
from pytorch_pretrained_bert import BertForMaskedLM
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
for param in model.bert.parameters():
    param.requires_grad = False
    
model.cuda()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
   

In [100]:
train_masks = create_masks(train_tokens2ids)
val_masks = create_masks(val_tokens2ids)
test_masks = create_masks(test_tokens2ids)

In [101]:
print(len(train_masks))
print(type(train_masks))
print(type(train_masks[0]))
print(len(train_masks[0]))
print(len(train_masks[0][0]))
print(len(train_masks[1]))
print(len(train_masks[1][1]))

print(len(val_masks))

print(len(test_masks))

print(len(data_train_tokenized['SU4'][0]))

print(len(train_tokens2ids[0]))

2075
<class 'list'>
<class 'numpy.ndarray'>
11
512
13
512
519
648
11
11


In [102]:
# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [112]:
train_labels = list(data_train_tokenized['SU4'])
val_labels = list(data_val_tokenized['SU4'])
print(len(train_labels))
print(len(val_labels))

train_tokens2ids = list(train_tokens2ids)
val_tokens2ids= list(val_tokens2ids)
print(len(train_tokens2ids[0]))
print(len(val_tokens2ids))

train_masks = list(train_masks)
val_masks= list(val_masks)
print(len(train_masks))
print(len(val_masks))

2075
519
11
519
2075
519


In [113]:
print(train_labels[:3])
print(train_tokens2ids[:3])
print(train_masks[:3])

print(type(train_labels))
print(type(train_tokens2ids))
print(type(train_masks))

print(len(train_labels))
print(len(train_tokens2ids))
print(len(train_masks))





[[0.15, 0.06383, 0.05714, 0.033710000000000004, 0.03077, 0.02, 0.0198, 0.01887, 0.01163, 0.0, 0.0], [0.39823, 0.37069, 0.28261, 0.27174, 0.21641, 0.12987, 0.09346, 0.08434, 0.07042000000000001, 0.02994, 0.02703, 0.02521, 0.0], [0.27273000000000003, 0.27273000000000003, 0.21267, 0.15859, 0.14331, 0.13618, 0.13546, 0.10593, 0.09616, 0.08867, 0.07142000000000001, 0.07142000000000001, 0.06225, 0.05759, 0.05586, 0.05154, 0.04959, 0.04946, 0.04721, 0.04396, 0.042460000000000005, 0.03846, 0.0367, 0.03665, 0.03553, 0.033839999999999995, 0.02857, 0.02747, 0.02358, 0.02, 0.01596, 0.005]]
[array([[  101,  6643,  8197, ...,     0,     0,     0],
       [  101, 17928, 24412, ...,     0,     0,     0],
       [  101,  2119, 19117, ...,     0,     0,     0],
       ...,
       [  101,  2057,  2179, ...,     0,     0,     0],
       [  101,  6643,  8197, ...,     0,     0,     0],
       [  101,  2023,  3319, ...,     0,     0,     0]]), array([[  101,  9808,  2618, ...,     0,     0,     0],
       [

In [125]:
def convert2array(givenlist):
  return np.array(givenlist)

In [126]:
train_tokens2ids_nparr = convert2array(train_tokens2ids)
val_tokens2ids_nparr = convert2array(val_tokens2ids)

train_masks_nparr = convert2array(train_masks)
val_masks_nparr = convert2array(val_masks)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [127]:
print(type(train_masks_nparr))
print(train_masks_nparr.shape)
print(train_masks_nparr[:3])

<class 'numpy.ndarray'>
(2075,)
[array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])
 array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])
 array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])]


In [135]:
def labellist2arr(givenlist):
  labels = []
  for i in range(len(givenlist)):
    label = np.array(givenlist[i])
    labels.append(label)
  return labels

In [142]:
train_labels_nparr = labellist2arr(train_labels)
val_labels_nparr = labellist2arr(val_labels)

In [143]:
def convert2tensor(givenarray):
  masks_tensors = []
  for i in range(len(givenarray)):
    tensor_mask = torch.from_numpy(givenarray[i])
    masks_tensors.append(tensor_mask)
  return masks_tensors

  

In [144]:
train_inputs_tensors = convert2tensor(train_tokens2ids_nparr)
val_inputs_tensors = convert2tensor(val_tokens2ids_nparr)

In [161]:
print(type(train_inputs_tensors))
print(type(train_inputs_tensors[0][0]))
print(train_inputs_tensors[0])

<class 'list'>
<class 'torch.Tensor'>
tensor([[  101,  6643,  8197,  ...,     0,     0,     0],
        [  101, 17928, 24412,  ...,     0,     0,     0],
        [  101,  2119, 19117,  ...,     0,     0,     0],
        ...,
        [  101,  2057,  2179,  ...,     0,     0,     0],
        [  101,  6643,  8197,  ...,     0,     0,     0],
        [  101,  2023,  3319,  ...,     0,     0,     0]])


In [146]:
train_masks_tensors = convert2tensor(train_masks_nparr)
val_masks_tensors = convert2tensor(val_masks_nparr)

In [159]:
print(type(train_masks_tensors))
print(type(train_masks_tensors[0]))
print(train_masks_tensors[0])

<class 'list'>
<class 'torch.Tensor'>
tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]], dtype=torch.float64)


In [149]:
train_labels_tensors = convert2tensor(train_labels_nparr)
val_labels_tensors = convert2tensor(val_labels_nparr)

In [174]:
print(type(train_labels_tensors))
print(type(train_labels_tensors[0]))
print(train_labels_tensors[0].shape)
print(train_inputs_tensors[0].shape)
print(train_masks_tensors[0].shape)

<class 'list'>
<class 'torch.Tensor'>
torch.Size([11])
torch.Size([11, 512])
torch.Size([11, 512])


In [ ]:
#train the model

# Select a batch size for training. 
batch_size = 32